## Creating an index and populating it with documents using Milvus and Nomic AI Embeddings

Simple example on how to ingest PDF documents, then web pages content into a Milvus VectorStore. . In this example, the embeddings are the fully open source ones released by NomicAI, [nomic-embed-text-v1](https://huggingface.co/nomic-ai/nomic-embed-text-v1).

As described in [this blog post](https://blog.nomic.ai/posts/nomic-embed-text-v1), those embeddings feature a "8192 context-length that outperforms OpenAI Ada-002 and text-embedding-3-small on both short and long context tasks". In additions, they are:

- Open source
- Open data
- Open training code
- Fully reproducible and auditable

### Requirements

- A **Milvus** vector database, set up according to [these instructions](../../../vector-databases/milvus/README.md).

- Connection credentials to Milvus must be available as environment variables:

  - `MILVUS_USERNAME`

  - `MILVUS_PASSWORD`

- Update the **MILVUS_HOST**, **MILVUS_PORT**, and **MILVUS_COLLECTION** in this notebook to match your deployment settings.

### Needed packages and imports

In [1]:
!pip install -q einops==0.7.0 langchain==0.1.9 pypdf==4.0.2 pymilvus==2.3.6 sentence-transformers==2.4.0

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
kfp 1.8.22 requires requests-toolbelt<1,>=0.8.0, but you have requests-toolbelt 1.0.0 which is incompatible.


In [2]:
import requests
import os
from langchain.document_loaders import PyPDFDirectoryLoader, WebBaseLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import Milvus

### Base parameters, the Milvus connection info

In [3]:
# Replace values according to your Milvus deployment
MILVUS_HOST = "milvus-service"
# MILVUS_HOST = "milvus-service.<your-namespace>.svc.cluster.local"
MILVUS_PORT = 19530
MILVUS_USERNAME = os.getenv('MILVUS_USERNAME')
MILVUS_PASSWORD = os.getenv('MILVUS_PASSWORD')
MILVUS_COLLECTION = "collection_nomicai_embeddings"

## Initial index creation and document ingestion

#### Download and load pdfs

In [4]:
product_version = "2.20"
documents = [
    "release_notes",
    "introduction_to_red_hat_openshift_ai",
    "getting_started_with_red_hat_openshift_ai_self-managed",
    "openshift_ai_tutorial_-_fraud_detection_example",
    "developing_a_model",
    "integrating_data_from_amazon_s3",
    "working_on_data_science_projects",
    "serving_models",
    "monitoring_data_science_models",
    "managing_users",
    "managing_resources",
    "installing_and_uninstalling_openshift_ai_self-managed",
    "installing_and_uninstalling_openshift_ai_self-managed_in_a_disconnected_environment",
    "upgrading_openshift_ai_self-managed",
    "upgrading_openshift_ai_self-managed_in_a_disconnected_environment",   
]

pdfs = [f"https://access.redhat.com/documentation/en-us/red_hat_openshift_ai_self-managed/{product_version}/pdf/{doc}/red_hat_openshift_ai_self-managed-{product_version}-{doc}-en-us.pdf" for doc in documents]
pdfs_to_urls = {f"red_hat_openshift_ai_self-managed-{product_version}-{doc}-en-us": f"https://access.redhat.com/documentation/en-us/red_hat_openshift_ai_self-managed/{product_version}/html-single/{doc}/index" for doc in documents}

In [5]:
docs_dir = f"rhoai-doc-{product_version}"

if not os.path.exists(docs_dir):
    os.mkdir(docs_dir)

for pdf in pdfs:
    try:
        response = requests.get(pdf)
    except:
        print(f"Skipped {pdf}")
        continue
    if response.status_code!=200:
        print(f"Skipped {pdf}")
        continue  
    with open(f"{docs_dir}/{pdf.split('/')[-1]}", 'wb') as f:
        f.write(response.content)

Skipped https://access.redhat.com/documentation/en-us/red_hat_openshift_ai_self-managed/2.20/pdf/developing_a_model/red_hat_openshift_ai_self-managed-2.20-developing_a_model-en-us.pdf
Skipped https://access.redhat.com/documentation/en-us/red_hat_openshift_ai_self-managed/2.20/pdf/integrating_data_from_amazon_s3/red_hat_openshift_ai_self-managed-2.20-integrating_data_from_amazon_s3-en-us.pdf
Skipped https://access.redhat.com/documentation/en-us/red_hat_openshift_ai_self-managed/2.20/pdf/managing_users/red_hat_openshift_ai_self-managed-2.20-managing_users-en-us.pdf


In [6]:
pdf_folder_path = f"./rhoai-doc-{product_version}"

pdf_loader = PyPDFDirectoryLoader(pdf_folder_path)
pdf_docs = pdf_loader.load()

#### Inject metadata

In [7]:
from pathlib import Path

for doc in pdf_docs:
    doc.metadata["source"] = pdfs_to_urls[Path(doc.metadata["source"]).stem]

#### Load websites

In [8]:
websites = [
    "https://ai-on-openshift.io/getting-started/openshift/",
    "https://ai-on-openshift.io/getting-started/opendatahub/",
    "https://ai-on-openshift.io/getting-started/openshift-ai/",
    "https://ai-on-openshift.io/odh-rhoai/configuration/",
    "https://ai-on-openshift.io/odh-rhoai/custom-notebooks/",
    "https://ai-on-openshift.io/odh-rhoai/nvidia-gpus/",
    "https://ai-on-openshift.io/odh-rhoai/custom-runtime-triton/",
    "https://ai-on-openshift.io/odh-rhoai/openshift-group-management/",
    "https://ai-on-openshift.io/tools-and-applications/minio/minio/",
    "https://access.redhat.com/articles/7047935",
    "https://access.redhat.com/articles/rhoai-supported-configs",
]

In [9]:
website_loader = WebBaseLoader(websites)
website_docs = website_loader.load()

#### Merge both types of docs

In [10]:
docs = pdf_docs + website_docs

#### Split documents into chunks with some overlap

In [11]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1024,
                                               chunk_overlap=40)
all_splits = text_splitter.split_documents(docs)
all_splits[0]

Document(page_content='Red Hat OpenShift AI Self-Managed\n2.20\nOpenShift AI tutorial - Fraud detection\nexample\nUse OpenShift AI to train an example model in JupyterLab, deploy the model, and\nrefine the model by using automated pipelines\nLast Updated: 2025-05-16', metadata={'source': 'https://access.redhat.com/documentation/en-us/red_hat_openshift_ai_self-managed/2.20/html-single/openshift_ai_tutorial_-_fraud_detection_example/index', 'page': 0})

#### Create the index and ingest the documents

In [12]:
# If you want to use a GPU, you can add the 'device': 'cuda' argument and provided you have used GPU Accelerator on your Workbench.
# model_kwargs = {'trust_remote_code': True, 'device': 'cuda'}
model_kwargs = {'trust_remote_code': True}

embeddings = HuggingFaceEmbeddings(
    model_name="nomic-ai/nomic-embed-text-v1",
    model_kwargs=model_kwargs,
    show_progress=True
)


db = Milvus(
    embedding_function=embeddings,
    connection_args={"host": MILVUS_HOST, "port": MILVUS_PORT, "user": MILVUS_USERNAME, "password": MILVUS_PASSWORD},
    collection_name=MILVUS_COLLECTION,
    metadata_field="metadata",
    text_field="page_content",
    auto_id=True,
    drop_old=True
    )

You try to use a model that was created with version 2.4.0.dev0, however, your version is 2.4.0. This might cause unexpected behavior or errors. In that case, try to update to the latest version.



/opt/app-root/lib64/python3.9/site-packages/networkx/utils/backends.py:135: RuntimeWarning: networkx backend defined more than once: nx-loopback
  backends.update(_get_backends("networkx.backends"))
2025-05-31 02:58:07.796583: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-05-31 02:58:08.998306: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
<All keys matched successfully>
/opt/app-root/lib64/python3.9/site-packages/pymilvus/client/__init__.py:6: UserWarning: pkg_resources is deprecated as an API.

In [13]:
db.add_documents(all_splits)

Batches:   0%|          | 0/44 [00:00<?, ?it/s]

[458399329430040290,
 458399329430040291,
 458399329430040292,
 458399329430040293,
 458399329430040294,
 458399329430040295,
 458399329430040296,
 458399329430040297,
 458399329430040298,
 458399329430040299,
 458399329430040300,
 458399329430040301,
 458399329430040302,
 458399329430040303,
 458399329430040304,
 458399329430040305,
 458399329430040306,
 458399329430040307,
 458399329430040308,
 458399329430040309,
 458399329430040310,
 458399329430040311,
 458399329430040312,
 458399329430040313,
 458399329430040314,
 458399329430040315,
 458399329430040316,
 458399329430040317,
 458399329430040318,
 458399329430040319,
 458399329430040320,
 458399329430040321,
 458399329430040322,
 458399329430040323,
 458399329430040324,
 458399329430040325,
 458399329430040326,
 458399329430040327,
 458399329430040328,
 458399329430040329,
 458399329430040330,
 458399329430040331,
 458399329430040332,
 458399329430040333,
 458399329430040334,
 458399329430040335,
 458399329430040336,
 458399329430

#### Test query

In [14]:
query = "How can I work with GPU and taints in OpenShift AI?"
docs_with_score = db.similarity_search_with_score(query)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [15]:
for doc, score in docs_with_score:
    print("-" * 80)
    print("Score: ", score)
    print(doc.page_content)
    print("-" * 80)

--------------------------------------------------------------------------------
Score:  0.4466226100921631
But don't worry, OpenShift AI and Open Data Hub take care of this part for you when you launch notebooks, workbenches, model servers, or pipeline runtimes!
Installation
Here is the documentation you can follow:

OpenShift AI documentation
NVIDIA documentation (more detailed)

Advanced configuration
Working with taints
In many cases, you will want to restrict access to GPUs, or be able to provide choice between different types of GPUs: simply stating "I want a GPU" is not enough. Also, if you want to make sure that only the Pods requiring GPUs end up on GPU-enabled nodes (and not other Pods that just end up being there at random because that's how Kubernetes works...), you're at the right place!
The only supported method at the moment to achieve this is to taint nodes, then apply tolerations on the Pods depending on where you want them scheduled. If you don't pay close attention t